# Blood Pressure Over Time

This notebook generates a chart of blood pressure readings over time from blood pressure data held in the Health Tracker database and retrieved via the Health Tracker Web Service.

It requires a copy of "api.ipynb" in the same folder, as this defines functions used to access the web service.

To use it, set the credentials, user details and chart export format in the first code cell before running the notebook.

In [ ]:
# Credentials for the user to log in as
username = ""
password = ""

# Details for the person to report for
firstnames = ""
surname = ""

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
export_format = "PNG"

In [ ]:
# Base URL for the Health Tracker service
url = "http://localhost:8099"

# Limits for high and low blood pressure readings
systolic_threshold = 140
diastolic_threshold = 90

# Threshold used to identify gaps in the data and segment it for plotting
gap_threshold = 7

In [ ]:
%run api.ipynb

In [ ]:
from datetime import date, datetime, timedelta

# Retrieve all-time blood pressure readings
start = date(1900, 1, 1)
end = datetime.now() + timedelta(days=1)

# Log in to the service, get the person ID and retrieve the blood pressure readings
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_daily_blood_pressure_measurements(url, token, person_id, start, end)

# Preview the data
df.head()

In [ ]:
import pandas as pd

# Make the data "gap aware" rather than trying to draw nonsensical lines across gaps. This is done
# by identifying gaps greather than the threshold in length and breaking the data into segments that
# can then be plotted separately
gap_threshold = pd.Timedelta(days=gap_threshold)
df["gap"] = df["date"].diff() > gap_threshold
df["segment"] = df["gap"].cumsum()

# Preview the data
df.head()

In [ ]:
import matplotlib.pyplot as plt

def plot_segments(df, column, color, label):
    for _, segment_df in df.groupby("segment"):
        # Need at least 2 points in a segment to draw the line
        if len(segment_df) < 2:
            continue

        # Plot the segment
        plt.plot(segment_df["date"], segment_df[column], color=color, label=label)
        label = None

In [ ]:
import pandas as pd
from pathlib import Path

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Define the export file name
export_file_name = "Blood-Pressure-Over-Time"
output_path = export_folder_path / f"{export_file_name}.xlsx"

# Create a Pandas Excel writer
with pd.ExcelWriter(output_path.absolute(), engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Blood Pressure", index=False)

In [ ]:
import matplotlib.pyplot as plt

# Plot systolic and diastolic
plt.figure(figsize=(12, 6))
plot_segments(df, "systolic", "red", "Systolic")
plot_segments(df, "diastolic", "blue", "Diastolic")

# Draw the hypertension threshold lines
plt.axhline(systolic_threshold, color="red", linestyle="--", linewidth=2, alpha=0.7, label="Systolic Upper Threshold")
plt.axhline(diastolic_threshold, color="blue", linestyle="--", linewidth=2, alpha=0.7, label="Diastolic Upper Threshold")

# Set title and labels
plt.title("Blood Pressure Over Time")
plt.xlabel("Date")
plt.ylabel("Blood Pressure (mmHg)")
plt.grid(True)

# Move legend below the chart
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.1), ncol=2)

plt.tight_layout()

# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()
